In [2]:
import json
import os
import glob
import re
import pandas as pd
from os import makedirs
from os.path import join, exists
from pathlib import Path
from datetime import datetime, date, timedelta
from pathlib import Path

In [3]:
def combine_data(year, cand, path, target_path, include_hd):
    
    tw_data = pd.read_csv(join(path, cand + '_twitter.csv'))
    tn_data = pd.read_csv(join(path,  cand + '_trends.csv'))
    df = []
    
    if include_hd == True:
        hd_data = pd.read_csv(join(path, cand + '_headlines.csv'))
        df = pd.merge(hd_data, tw_data, how='right', on=['date', 'date'], suffixes=('_h', '_t'))
        df = pd.merge(tn_data, df, how='right', on=['date', 'date'], suffixes=('_tn', '_a'))
        
        df = drop_unnamed_columns(df)
        df = df.fillna(df.mean())
        
        print(df.columns)
        
        df['score']=df['score'].replace(0, df['score'].mean())
        df['score_mn']=df['score_mn'].replace(0, df['score_mn'].mean())
        
        col = df.loc[: , df.columns[2]: df.columns[len(df.columns) - 1]]
        df['score_mean'] = col.mean(axis=1)
        df['final_score'] = df[df.columns[1]] * df['score_mean']
    else:
        #df = pd.merge(hd_data, tw_data, how='right', on=['date', 'date'], suffixes=('_h', '_t'))
        df = pd.merge(tn_data, tw_data, how='right', on=['date', 'date'])
        #df = tw_data
        df = drop_unnamed_columns(df)
        df = df.fillna(df.mean())
        
        print(df.columns)
        
        scores = []
        
        for index, row in df.iterrows():
            if row['score_y'] >= 1:
                scores.append((row['score_x'] * row['score_y']) + 100)
            else:
                scores.append((-1 * row['score_x'] * row['score_y']) + 100)
            
        #print(scores)
        
        df['final_score'] = pd.DataFrame(scores)
        
        #df['final_score'] = df[df.columns[1]] * df[df.columns[2]]
        
        #print(tw_data.columns)
        
        #df['final_score'] = tw_data[tw_data.columns[3]]
    
    df2 = df[['date', 'final_score']]
    
    os.makedirs(target_path, exist_ok=True)
    
    df2.to_csv(join(target_path, cand + '_combined.csv'), index=False)
    

In [4]:
def drop_unnamed_columns(df):
    cols = df.columns
    for c in cols:
        if c == 'Unnamed: 0' or c == 'Unnamed: 0.1' or c == 'Unnamed: 0_x' or c == 'Unnamed: 0_a' \
        or c == 'Unnamed: 0_y' or c == 'headline' or c == 'Unnamed: 0_tn':
            df = df.drop([c], axis=1)
    
    return df

In [5]:
def combine_all_news_hd(cand, path):
    hd_c_data = pd.read_csv(join(path, cand + '_headlines_cnn.csv'))
    hd_c_data = drop_unnamed_columns(hd_c_data)
    hd_f_data = pd.read_csv(join(path, cand + '_headlines_fox-news.csv'))
    hd_f_data = drop_unnamed_columns(hd_f_data)
    #hd_g_data = pd.read_csv(join(path, cand + '_headlines_guardian.csv'))
    #hd_g_data = drop_unnamed_columns(hd_g_data)
    
    
    df = pd.merge(hd_c_data, hd_f_data, how='inner', on=['date', 'date'], suffixes=('_c', '_f'))
    #df = pd.merge(hd_g_data, df, how='right', on=['date', 'date'], suffixes=('_g', '_a'))
    
    df = df.fillna(df.mean())
    
    col = df.loc[: , "score_c":"score_f"]
    df['score_mn'] = col.mean(axis=1)
    
    df2 = df[['date', 'score_mn']]
    
    df2.to_csv(join(path, cand + '_headlines.csv'), index=False)

In [6]:
def get_cand_by_year(yr):
    if yr == '2012':
        return 'obama', 'romney'
    elif yr == '2016':
        return 'hillary', 'trump'
    else:
        return 'biden', 'trump'

In [7]:
def combine_twt_data():
    for yr in ['2012', '2016', '2020']:
        for tp in ['candidate', 'economy', 'environment', 'health', 'immigration']:

            cand1, cand2 = get_cand_by_year(yr)
            
            combine_data('2020', cand1, join('data22', tp, yr), join('data23', tp, yr), False)
            combine_data('2020', cand2, join('data22', tp, yr), join('data23', tp, yr), False)

In [8]:
combine_twt_data()

Index(['Unnamed: 0.1_x', 'date', 'score_x', 'Unnamed: 0.1_y', 'score_y'], dtype='object')
Index(['Unnamed: 0.1_x', 'date', 'score_x', 'Unnamed: 0.1_y', 'score_y'], dtype='object')
Index(['Unnamed: 0.1_x', 'date', 'score_x', 'Unnamed: 0.1_y', 'score_y'], dtype='object')
Index(['Unnamed: 0.1_x', 'date', 'score_x', 'Unnamed: 0.1_y', 'score_y'], dtype='object')
Index(['Unnamed: 0.1_x', 'date', 'score_x', 'Unnamed: 0.1_y', 'score_y'], dtype='object')
Index(['Unnamed: 0.1_x', 'date', 'score_x', 'Unnamed: 0.1_y', 'score_y'], dtype='object')
Index(['Unnamed: 0.1_x', 'date', 'score_x', 'Unnamed: 0.1_y', 'score_y'], dtype='object')
Index(['Unnamed: 0.1_x', 'date', 'score_x', 'Unnamed: 0.1_y', 'score_y'], dtype='object')
Index(['Unnamed: 0.1_x', 'date', 'score_x', 'Unnamed: 0.1_y', 'score_y'], dtype='object')
Index(['Unnamed: 0.1_x', 'date', 'score_x', 'Unnamed: 0.1_y', 'score_y'], dtype='object')
Index(['Unnamed: 0.1_x', 'date', 'score_x', 'Unnamed: 0.1_y', 'score_y'], dtype='object')
Index(['Un

In [59]:
combine_data('2012', 'obama', join('data2', 'candidate', '2012'), join('data3', 'candidate', '2012'), False)
combine_data('2012', 'romney', join('data2', 'candidate', '2012'), join('data3', 'candidate', '2012'), False)

Index(['date', 'score_x', 'score_y'], dtype='object')
1.0447154471544715
1.0221729490022171
0.9934210526315792
-2.235197368421053
0.997872340425532
-5.613031914893617
1.0573770491803278
0.9979253112033196
-11.143499308437066
1.0084388185654007
1.021097046413502
0.9936440677966102
-6.5414901129943495
0.9958762886597938
-6.224226804123711
1.033402922755741
1.0255863539445629
1.0126315789473683
1.045360824742268
1.028322440087146
1.0538302277432712
1.0248962655601659
1.028688524590164
[107.61771680216802, 106.47376201034737, 97.76480263157895, 94.38696808510639, 105.24282786885246, 88.85650069156293, 106.76494374120956, 105.44585091420535, 93.45850988700565, 93.77577319587628, 105.85594989561586, 106.40991471215352, 105.86482456140351, 108.84201030927835, 107.66957153231662, 113.56806418219462, 114.43395573997233, 115.90181010928963]
Index(['date', 'score_x', 'score_y'], dtype='object')
1.0245398773006136
1.0173160173160174
1.0376569037656904
1.0125260960334028
0.9814049586776858
-9.99248

In [60]:
combine_data('2016', 'trump', join('data2', 'candidate', '2016'), join('data3', 'candidate', '2016'), False)
combine_data('2016', 'hillary', join('data2', 'candidate', '2016'), join('data3', 'candidate', '2016'), False)

Index(['date', 'score_x', 'score_y'], dtype='object')
0.8840579710144928
-6.225241545893721
0.8489795918367347
-3.4312925170068027
0.9157667386609072
-6.868250539956804
0.9850746268656716
-6.402985074626866
0.9764957264957264
-8.056089743589743
0.98744769874477
-8.928172942817294
0.9561586638830898
-4.023834377174669
0.9807280513918628
-4.535867237687365
0.9592274678111588
-1.838519313304721
0.9648351648351648
-5.70860805860806
0.9849137931034484
-7.017510775862069
0.971677559912854
-4.7369281045751634
0.9957446808510638
-10.745744680851061
0.9978858350951374
-3.9083861874559545
0.9917184265010353
-5.041235334713596
0.9658848614072496
-5.513592750533049
0.9872068230277186
-3.784292821606255
0.9130434782608696
-6.201086956521741
0.9488272921108742
-6.7999289267946
1.0426439232409381
0.9892703862660944
-6.59513590844063
1.0086767895878526
0.9785407725321889
-5.830472103004291
0.9747368421052632
-5.848421052631579
0.9978586723768736
-6.278194147037831
0.9626865671641792
-9.586753731343284

In [61]:
#combine_all_news_hd('trump', join('data', 'candidate', '2020'))
#combine_all_news_hd('biden', join('data', 'candidate', '2020'))

In [62]:
combine_data('2020', 'trump', join('data2', 'candidate', '2020'), join('data3', 'candidate', '2020'), False)
combine_data('2020', 'biden', join('data2', 'candidate', '2020'), join('data3', 'candidate', '2020'), False)

Index(['date', 'score_x', 'score_y'], dtype='object')
0.9516539440203562
-32.35623409669211
0.8846153846153846
-42.24038461538461
0.9556650246305419
-39.0628078817734
0.9276807980049876
-46.03615960099751
0.9334811529933481
-67.4440133037694
0.8868894601542416
-77.2702442159383
0.9871134020618556
-61.44780927835051
0.9775561097256856
-51.68827930174563
0.9606879606879608
-52.71775184275185
0.9036458333333334
-45.52115885416667
0.9025
-39.033125
0.9527363184079602
-31.916666666666668
0.9174757281553398
-30.96480582524272
0.9951690821256038
-34.08454106280193
0.9101941747572816
-30.832827669902915
0.9580246913580248
-34.60864197530864
1.0609756097560976
1.0465686274509804
0.9217171717171716
-42.1685606060606
0.8944844124700241
-34.77308153477219
0.9638554216867472
-34.819277108433745
0.9659367396593674
-41.052311435523116
0.95873786407767
-43.26304611650486
0.9498861047835992
-63.40489749430525
0.9746192893401016
-94.1725888324873
1.0165876777251184
0.9922279792746114
-51.34779792746114


In [46]:
for tp in ['economy', 'environment', 'health', 'immigration', 'party']:
    combine_all_news_hd('trump', join('data', tp))
    combine_all_news_hd('biden', join('data', tp))
    
    combine_data('2020', 'trump', join('data', tp), join('data2', tp), True)
    combine_data('2020', 'biden', join('data', tp), join('data2', tp), True)

Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
